In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

# set the filepath and load in a shapefile 
fp = "../cloropleth_python/IDN_adm/IDN_adm1.shp"
map_df = gpd.read_file(fp)
# check the GeoDataframe
map_df.head()

,ID_0,ISO,NAME_0,ID_1,NAME_1,TYPE_1,ENGTYPE_1,NL_NAME_1,VARNAME_1,geometry
0,106,IDN,Indonesia,1,Aceh,Propinsi,Autonomous Province,None,Achin|Atjeh|Nanggroe Aceh Darussalam,"MULTIPOLYGON (((97.38493 1.97713, 97.38396 1.9..."
1,106,IDN,Indonesia,2,Bali,Propinsi,Province,None,Penida|Lembongan|Ceningan|Menjangan,"MULTIPOLYGON (((115.62988 -8.77599, 115.63037 ..."
2,106,IDN,Indonesia,3,Bangka-Belitung,Propinsi,Province,None,Babel|Kepulauan Bangka Belitung,"MULTIPOLYGON (((108.06951 -3.80305, 108.06826 ..."
3,106,IDN,Indonesia,4,Banten,Propinsi,Province,None,None,"MULTIPOLYGON (((105.54861 -6.99472, 105.54889 ..."
4,106,IDN,Indonesia,5,Bengkulu,Propinsi,Province,None,Bencoolen|Benkoelen|Benkulen,"MULTIPOLYGON (((102.38627 -5.47112, 102.38682 ..."


In [2]:
# plt.rcParams['figure.figsize'] = [20, 13] #height, width
# map_df.plot()

In [3]:
map_df.dtypes

ID_0            int64
ISO            object
NAME_0         object
ID_1            int64
NAME_1         object
TYPE_1         object
ENGTYPE_1      object
NL_NAME_1      object
VARNAME_1      object
geometry     geometry
dtype: object

In [4]:
map_df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [84]:
#Load the list of addresses and their Lat Longs
list_location = pd.read_csv('metro_stores.csv', sep=";")
#changing to a GeoDataFrame to create geometry series
list_location_gp = gpd.GeoDataFrame(list_location, geometry=gpd.points_from_xy(list_location.Longitude,list_location.Latitude))

list_location_gp.head()

,Address,Latitude,Longitude,geometry
0,"Jl. Metro Pondok Indah, Jakarta 12310, Indonesia",-6.275405,106.782070,POINT (106.78207 -6.27540)
1,"Jl. Asia Afrika No. 8, Jakarta 10270",-6.225578,106.799035,POINT (106.79904 -6.22558)
2,"Jl. Gatot Subroto No. 289, Bandung 40273",-6.924923,107.636701,POINT (107.63670 -6.92492)
3,"Jl. Letjen S. Parman Kav. 21, Slipi, Jakarta 1...",-6.179206,106.793160,POINT (106.79316 -6.17921)
4,"Jl. H.M. Daeng Patompo, Metro Tanjung Bunga, M...",-5.166200,119.391700,POINT (119.39170 -5.16620)


In [88]:
list_location_gp['PROVINCE'] = ''
for idx in range(map_df.shape[0]):
    #For every untagged addresses, find if they reside within a province
    pip = list_location_gp.within(map_df.loc[idx, 'geometry'])
    if pip.sum() > 0: #we found where some of the addresses reside at map_df.loc[idx]
        list_location_gp.loc[pip, 'PROVINCE']  = map_df.loc[idx, 'NAME_1']

list_location_gp

,Address,Latitude,Longitude,geometry,PROVINCE
0,"Jl. Metro Pondok Indah, Jakarta 12310, Indonesia",-6.275405,106.782070,POINT (106.78207 -6.27540),Jakarta Raya
1,"Jl. Asia Afrika No. 8, Jakarta 10270",-6.225578,106.799035,POINT (106.79904 -6.22558),Jakarta Raya
2,"Jl. Gatot Subroto No. 289, Bandung 40273",-6.924923,107.636701,POINT (107.63670 -6.92492),Jawa Barat
3,"Jl. Letjen S. Parman Kav. 21, Slipi, Jakarta 1...",-6.179206,106.793160,POINT (106.79316 -6.17921),Jakarta Raya
4,"Jl. H.M. Daeng Patompo, Metro Tanjung Bunga, M...",-5.166200,119.391700,POINT (119.39170 -5.16620),Sulawesi Selatan
5,"Jl. Sultan Iskandar Muda, Gandaria, Kebayoran ...",-6.242726,106.783286,POINT (106.78329 -6.24273),Jakarta Raya
6,"Jl. Mayjend Sungkono Kav. 89, Surabaya, 60189,...",-7.293022,112.719335,POINT (112.71934 -7.29302),Jawa Timur
7,"Jl. IR. Soekarno, Madegondo, Grogol, Sukoharjo...",-7.601382,110.813756,POINT (110.81376 -7.60138),Jawa Tengah
8,"Jl. Puri Agung, Puri Indah, Kembangan Selatan,...",-6.187837,106.733214,POINT (106.73321 -6.18784),Jakarta Raya
9,"Jl. Alternatif Cibubur, Harjamukti, Cimanggis,...",-6.375265,106.902028,POINT (106.90203 -6.37527),Jakarta Raya
